In [ ]:
#| default_exp auto

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from os import cpu_count
import torch

from ray import tune
from ray.tune.search.basic_variant import BasicVariantGenerator

from neuralforecast.common._base_auto import BaseAuto

from neuralforecast.models.rnn import RNN
from neuralforecast.models.lstm import LSTM
from neuralforecast.models.gru import GRU
from neuralforecast.models.dilated_rnn import DilatedRNN

from neuralforecast.models.mlp import MLP
from neuralforecast.models.nbeats import NBEATS
from neuralforecast.models.nhits import NHITS

from neuralforecast.losses.pytorch import MAE
from neuralforecast.losses.pytorch import MSE

In [ ]:
#| hide
from fastcore.test import test_eq
from nbdev.showdoc import show_doc

In [ ]:
#| hide
import logging
import warnings
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

# <span style="color:DarkOrange"> Models </span>

> NeuralForecast contains user-friendly implementations of neural forecasting models that allow for easy transition of computing capabilities (GPU/CPU), computation parallelization, and hyperparameter tuning. All the NeuralForecast models are "global" because we train them with all the series from the input pd.DataFrame data `Y_df`, yet the optimization objective is, momentarily, "univariate" as it does not consider the interaction between the output predictions across time series. Like the StatsForecast library, `core.NeuralForecast` allows you to explore collections of models efficiently and contains functions for convenient wrangling of input and output pd.DataFrames predictions.

# <span style="color:DarkBlue"> 1. Automatic Forecasting </span>

##  AutoRNN

In [ ]:
#| export
class AutoRNN(BaseAuto):
    
    default_config = {
        "input_size_multiplier": [1, 2, 3],
        "h": None,
        "state_hsize": tune.choice([100, 200, 300]),
        "n_layers": tune.randint(1, 4),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": tune.choice([MAE(), MSE()]),
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False):

        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            del config["input_size_multiplier"]

        super(AutoRNN, self).__init__(
              cls_model=RNN, 
              h=h,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples, 
              cpus=cpus,
              gpus=gpus,
              verbose=verbose
        )

In [ ]:
%%capture
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengersDF as Y_df

# Split train/test and declare time series dataset
Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31']   # 12 test
dataset, *_ = TimeSeriesDataset.from_df(Y_train_df)

# Use your own config or AutoRNN.default_config
config = dict(max_steps=10, input_size=12, state_hsize=256)
model = AutoRNN(h=12, config=config, num_samples=1, cpus=1)

model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

In [ ]:
# Plotting predictions
Y_plot_df = Y_test_df.copy()
Y_plot_df['AutoRNN'] = y_hat
pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

##  AutoLSTM

In [ ]:
#| export
class AutoLSTM(BaseAuto):
    
    default_config = {
        "input_size_multiplier": [1, 2, 3],
        "h": None,
        "state_hsize": tune.choice([100, 200, 300]),
        "n_layers": tune.randint(1, 4),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": tune.choice([MAE(), MSE()]),
        "random_seed": tune.randint(1, 20)
    }
    
    def __init__(self,
                 h,
                 config, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False):

        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            del config["input_size_multiplier"]

        super(AutoLSTM, self).__init__(
              cls_model=LSTM,
              h=h,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              cpus=cpus,
              gpus=gpus,
              verbose=verbose
        )

In [ ]:
%%capture
# Use your own config or AutoLSTM.default_config
config = dict(max_steps=10, input_size=12)
model = AutoLSTM(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

In [ ]:
#| hide
Y_test_df['AutoLSTM'] = y_hat
Y_plot_df = pd.concat([Y_train_df,
                       Y_test_df[['unique_id', 'ds', 'AutoLSTM']]])
pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

##  AutoGRU

In [ ]:
#| export
class AutoGRU(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3],
        "h": None,
        "state_hsize": tune.choice([100, 200, 300]),
        "n_layers": tune.randint(1, 4),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": tune.choice([MAE(), MSE()]),
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 config,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            del config["input_size_multiplier"]

        super(AutoGRU, self).__init__(
              cls_model=GRU,
              h=h,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples, 
              cpus=cpus,
              gpus=gpus,
              verbose=verbose
        )

In [ ]:
%%capture
# Use your own config or AutoGRU.default_config
config = dict(max_steps=10, input_size=12)
model = AutoGRU(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

In [ ]:
#| hide
Y_test_df['AutoGRU'] = y_hat
Y_plot_df = pd.concat([Y_train_df,
                       Y_test_df[['unique_id', 'ds', 'AutoGRU']]])
pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

##  AutoDilatedRNN

In [ ]:
#| export
class AutoDilatedRNN(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3],
        "h": None,
        "cell_type": tune.choice(['LSTM', 'GRU']),
        "state_hsize": tune.choice([100, 200, 300]),
        "dilations": tune.choice([ [[1, 2], [4, 8]], [[1, 2, 4, 8]] ]),
        "add_nl_layer": tune.choice([True, False]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": tune.choice([MAE(), MSE()]),
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 config, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            del config["input_size_multiplier"]

        super(AutoDilatedRNN, self).__init__(
              cls_model=DilatedRNN,
              h=h,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              cpus=cpus,
              gpus=gpus,
              verbose=verbose
         )

In [ ]:
%%capture
# Use your own config or AutoDilatedRNN.default_config
config = dict(max_epochs=10, input_size=24)
model = AutoDilatedRNN(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

In [ ]:
#| hide
Y_test_df['AutoDilatedRNN'] = y_hat
Y_plot_df = pd.concat([Y_train_df,
                       Y_test_df[['unique_id', 'ds', 'AutoDilatedRNN']]])
pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

##  AutoMLP

In [ ]:
#| export
class AutoMLP(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice( [256, 512, 1024] ),
        "num_layers": tune.randint(2, 6),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "normalize": tune.choice([True, False]),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": tune.choice([MAE(), MSE()]),
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,                 
                 config, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False):

        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoMLP, self).__init__(
              cls_model=MLP,
              h=h,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples, 
              cpus=cpus,
              gpus=gpus,
              verbose=verbose
        )

In [ ]:
%%capture
# Use your own config or AutoMLP.default_config
config = dict(max_steps=10, input_size=12)
model = AutoMLP(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

In [ ]:
#| hide
Y_test_df['AutoMLP'] = y_hat
Y_plot_df = pd.concat([Y_train_df, 
                       Y_test_df[['unique_id', 'ds', 'AutoMLP']]])
pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

##  AutoNBEATS

In [ ]:
#| export
class AutoNBEATS(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "normalize": tune.choice([True, False]),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": tune.choice([MAE(), MSE()]),
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoNBEATS, self).__init__(
              cls_model=NBEATS, 
              h=h,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              cpus=cpus,
              gpus=gpus,
              verbose=verbose
        )

In [ ]:
%%capture
# Use your own config or AutoNBEATS.default_config
config = dict(max_steps=10, input_size=12)
model = AutoNBEATS(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

In [ ]:
#| hide
Y_plot_df = Y_test_df.copy()
Y_plot_df['AutoNBEATS'] = y_hat
pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

##  AutoNHITS

In [ ]:
#| export
class AutoNHITS(BaseAuto):

    default_config = {
       "input_size_multiplier": [1, 2, 3, 4, 5],
       "h": None,
       "n_pool_kernel_size": tune.choice([3*[1], 3*[2], 3*[4], 
                                          [8, 4, 1], [16, 8, 1]]),
       "n_freq_downsample": tune.choice([[168, 24, 1], [24, 12, 1], 
                                         [180, 60, 1], [60, 8, 1], 
                                         [40, 20, 1], [1, 1, 1]]),
       "learning_rate": tune.loguniform(1e-4, 1e-1),
       "normalize": tune.choice([True, False]),
       "max_steps": tune.choice([500, 1000]),
       "batch_size": tune.choice([32, 64, 128, 256]),
       "windows_batch_size": tune.choice([128, 256, 512, 1024]),
       "loss": tune.choice([MAE(), MSE()]),
       "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            
            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoNHITS, self).__init__(
              cls_model=NHITS, 
              h=h,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              cpus=cpus,
              gpus=gpus,
              verbose=verbose
        )

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, input_size=12)
model = AutoNHITS(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

In [ ]:
#| hide
Y_plot_df = Y_test_df.copy()
Y_plot_df['AutoNHITS'] = y_hat
pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

# <span style="color:DarkBlue"> 2. Base Models </span>

##  RNN

##  LSTM

##  GRU

##  DilatedRNN

##  MLP

##  NBEATS

##  NHITS

##  TFT